In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

#! mkdir ~/.kaggle
! cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d divg07/casia-20-image-tampering-detection-dataset

In [ ]:
!unzip casia-20-image-tampering-detection-dataset.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import cv2
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import joblib
import warnings
import math
import pickle as pk
import datetime
import albumentations
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from imageio import imread
import imageio
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers,Model,Sequential
from tensorflow.keras.layers import Dropout,Activation,InputLayer,LSTM,GRU,Bidirectional,TimeDistributed,Flatten,Dense,BatchNormalization,MaxPooling2D,Conv2D,Input,Concatenate,LeakyReLU
warnings.filterwarnings('ignore')
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

In [ ]:
masked_dict  = {'name':[],'height':[], 'width':[], 'channels':[]}
data_dict = {'name':[],'height':[], 'width':[], 'channels':[],'label':[]}

In [ ]:
for image in tqdm(os.listdir('/content/CASIA2/CASIA 2 Groundtruth')):
    if image.endswith('jpg') or image.endswith('png'):
        img = imread('/content/CASIA2/CASIA 2 Groundtruth'+'/'+image)
        if len(img.shape)==2:
            height, width = img.shape
            channels = 1
        else:
            height, width, channels = img.shape
        masked_dict['name'].append(image)
        masked_dict['height'].append(height)
        masked_dict['width'].append(width)
        masked_dict['channels'].append(channels)

In [ ]:
type(masked_dict)

In [ ]:
type(masked_dict["height"])

In [ ]:
for image in tqdm(os.listdir('/content/CASIA2/Tp')):
    if image.endswith('jpg') or image.endswith('png'):
        img = imread('/content/CASIA2/Tp'+'/'+image)
        if len(img.shape)==2:
            height, width = img.shape
            channels = 1
        else:
            height, width, channels = img.shape
        data_dict['name'].append(image)
        data_dict['height'].append(height)
        data_dict['width'].append(width)
        data_dict['channels'].append(channels)
        data_dict['label'].append('fake')

In [ ]:
for image in tqdm(os.listdir('/content/CASIA2/Au')):
    if image.endswith('jpg') or image.endswith('png'):
        img = imread('/content/CASIA2/Au'+'/'+image)
        if len(img.shape)==2:
            height, width = img.shape
            channels = 1
        else:
            height, width, channels = img.shape
        data_dict['name'].append(image)
        data_dict['height'].append(height)
        data_dict['width'].append(width)
        data_dict['channels'].append(channels)
        data_dict['label'].append('pristine')

In [ ]:
data_dict.keys()

In [ ]:
masked_dict.keys()

In [ ]:
print(data_dict['height'][0])

In [ ]:
data_df = pd.DataFrame.from_dict(data_dict)
masked_df = pd.DataFrame.from_dict(masked_dict)

In [ ]:
data_df.shape

In [ ]:
masked_df.shape

In [ ]:
data_df

In [ ]:
masked_df

In [ ]:
masked_df.head()

In [ ]:
data_df.head()

In [ ]:
data_df.duplicated().any()

In [ ]:
masked_df.duplicated().any()

In [ ]:
lables= data_df['label'].value_counts()
ax=lables.plot.bar(width=.8,title='Labels in Train Data', rot=0)
for i, v in lables.reset_index().iterrows():
    ax.text(i, v.label, v.label)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].set_title('PDF of FAKE Image heights')
sns.distplot(data_df[data_df['label']=='fake'].height,ax=ax[0])
ax[0].set_xlabel("Image height in pixels")
#sns.distplot(data_df[data_df['label']=='pristine'].height)
ax[0].set_xlabel("Image height in pixels")
sns.distplot(data_df[data_df['label']=='pristine'].height,ax=ax[1])
#sns.distplot(data_df[data_df['label']=='pristine'].height)
ax[1].set_xlabel("Image height in pixels")
ax[1].set_title('PDF of PRISTINE Image heights')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].set_title('PDF of FAKE Image widths')
sns.distplot(data_df[data_df['label']=='fake'].width,ax=ax[0])
ax[0].set_xlabel("Image widths in pixels")
ax[0].set_xlabel("Image widths in pixels")
sns.distplot(data_df[data_df['label']=='pristine'].width,ax=ax[1])
ax[1].set_xlabel("Image widths in pixels")
ax[1].set_title('PDF of PRISTINE Image widths')
plt.show()

In [ ]:
lables= data_df['channels'].value_counts()
ax=lables.plot.bar(width=.8,title='Channels in Images', rot=0)
for i, v in lables.reset_index().iterrows():
    ax.text(i, v.channels, v.channels)

In [ ]:
lables= masked_df['channels'].value_counts()
ax=lables.plot.bar(width=.8,title='Channels in Mask images', rot=0)
for i, v in lables.reset_index().iterrows():
    ax.text(i, v.channels, v.channels)

In [ ]:
plt.figure(figsize=(10,10))
image = imread('/content/CASIA2/Au'+'/'+(data_df[data_df.label=='pristine']['name']).iloc[120])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Apply Canny edge detection
edges = cv2.Canny(gray,100,200)
plt.imshow(image)
plt.title('A Sample from PRISTINE images')
plt.show()
plt.imshow(edges)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,15))
index = 15
image = imread('/content/CASIA2/Tp'+'/'+(data_df[data_df.label=='fake']['name']).iloc[index])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask_image = imread('/content/CASIA2/CASIA 2 Groundtruth'+'/'+(data_df[data_df.label=='fake']['name']).iloc[index].split('.')[0]+'_gt.png')
mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#     # Perform histogram equalization to improve contrast
equalized = cv2.equalizeHist(blurred)
edges = cv2.Canny(equalized,200,500)

ax[0].imshow(image)
ax[1].imshow(mask_image,cmap='gray')
ax[0].set_title('A Sample from FAKE images')
ax[1].set_title('Corresponding MASK for this FAKE image')
plt.imshow(edges)
plt.show()

In [ ]:
plt.imshow(edges)

In [ ]:
index = 300
fig, ax = plt.subplots(1,2,figsize=(15,15))
image = imread('/content/CASIA2/Tp'+'/'+(data_df[data_df.label=='fake']['name']).iloc[index])
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask_image = imread('/content/CASIA2/CASIA 2 Groundtruth'+'/'+(data_df[data_df.label=='fake']['name']).iloc[index].split('.')[0]+'_gt.png')
#mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
ax[0].imshow(image)
ax[1].imshow(mask_image,cmap='gray')
ax[0].set_title('A Sample from FAKE images')
ax[1].set_title('Corresponding MASK for this FAKE image')
plt.show()

In [ ]:
print("Total FAKE images in the data: {}".format(len(set(list(data_df[data_df.label=='fake']['name'])))))
print("Total availble Masks for the data: {}".format(len(set(list(masked_df['name'])))))

In [ ]:
print("Total pristine images in the data: {}".format(len(set(list(data_df[data_df.label=='pristine']['name'])))))

In [ ]:
fake_names = list(set([i.split('.')[0] for i in list(masked_df['name'])]) & set([i.replace('.jpg','_gt.png').split('.')[0] for i in list(data_df[data_df.label=='fake']['name'])]))


In [ ]:
len(fake_names)

In [ ]:
fake_names1 = list(set([i.split('.')[0] for i in list(masked_df['name'])]) & set([i.replace('.jpg','.png').split('.')[0] for i in list(data_df[data_df.label=='fake']['name'])]))


In [ ]:
fake_names1

In [ ]:
data_df.head()

In [ ]:
print(fake_names)

In [ ]:
print("Total intersection of Masks and FAKE images in the data: {}".format(len(fake_names)))


In [ ]:
delete_these = []
for image in fake_names:
    mask_img = cv2.imread('/content/CASIA2/CASIA 2 Groundtruth'+'/'+image+'.png')
    fake_img = cv2.imread('/content/CASIA2/Tp'+'/'+image.replace('_gt','')+'.jpg')
    if mask_img.shape[:2]!= fake_img.shape[:2]:
        delete_these.append(image)
#delete_these

In [ ]:
len(delete_these)

In [ ]:
fake_names=list(set(fake_names)-set(delete_these))
print("Total valid masks are :{}".format(len(fake_names)))

In [ ]:
fake_names1=[sub.replace('_gt', '') for sub in fake_names]

In [ ]:
dest_path='/content/binary_images1/'
# os.makedirs(dest_path)
bin_masks =[]
for mask in tqdm(fake_names):
    mask_img = cv2.imread('/content/CASIA2/CASIA 2 Groundtruth'+'/'+mask+'.png')[:,:,:1]
    #mask_img= cv2.cvtColor(mask_img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(mask_img,(5,5),0) #Adding gaussian blur
    ret,bin_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) #modifing the values to either 0 or 255
    cv2.imwrite(dest_path+mask+'binary.png',bin_mask)
    bin_masks.append(bin_mask)

In [ ]:
# index = 104
# fig, ax = plt.subplots(1,3,figsize=(20,20))
# image = imread('/content/CASIA2/Tp'+'/'+(data_df[data_df.label=='fake']['name']).iloc[index])
# #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# mask_image = imread('/content/CASIA2/CASIA 2 Groundtruth'+'/'+(data_df[data_df.label=='fake']['name']).iloc[index].split('.')[0]+'_gt.png')
# #mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
# bin_mask = imread('/content/binary_images1/'+(data_df[data_df.label=='fake']['name']).iloc[index].split('.')[0]+'_gtbinary.png')
# #bin_mask = cv2.cvtColor(bin_mask, cv2.COLOR_BGR2GRAY)
# ax[0].imshow(image)
# ax[1].imshow(mask_image,cmap='gray')
# ax[2].imshow(bin_mask,cmap='gray')
# ax[0].set_title('A Sample from FAKE images')
# ax[1].set_title('Corresponding MASK for this FAKE image')
# ax[2].set_title('BINARY-MASK for this FAKE image')
# plt.show()

In [ ]:
# data_df.head()

In [ ]:
fake_names_df = pd.DataFrame([i+'.jpg' for i in fake_names1],columns=['name'])
data_df_real = data_df[data_df.label == 'pristine']
data_df_fake = data_df[data_df.label == 'fake']
data_df_fake = data_df_fake.join(fake_names_df.set_index('name'), on='name',how='right')

In [ ]:
data_df_final = pd.concat([data_df_real, data_df_fake], ignore_index=True)

In [ ]:
data_df_fake.shape

In [ ]:
data_df_real.shape

In [ ]:
data_df_final.shape

In [ ]:
data_df_final.to_csv('data_df_final.csv',index=False)

In [ ]:
lables= data_df_final['label'].value_counts()
ax=lables.plot.bar(width=.8,title='Labels in Data after cleaning', rot=0)
for i, v in lables.reset_index().iterrows():
    ax.text(i, v.label, v.label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_df_final[['channels', 'height', 'name', 'width','label']],
                                                    data_df_final['label'],
                                                    test_size=0.2,
                                                    stratify=data_df_final['label'],
                                                    random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train[['channels', 'height', 'name', 'width','label']],
                                                  y_train, test_size=0.2, stratify=y_train,random_state=42)

In [ ]:
print("........Data splits before the Agumentation...........")
print("Images in Train data before sampling: {} ".format(X_train.shape[0]))
print("Images in Test data  before sampling: {} ".format(X_test.shape[0]))
print("Images in Validation data before sampling: {} ".format(X_val.shape[0]))

In [ ]:
X_train.to_csv('X_train.csv',index=False)
X_test.to_csv('X_test.csv',index=False)
X_val.to_csv('X_val.csv',index=False)


In [ ]:
X_train= pd.read_csv('X_train.csv')
X_test= pd.read_csv('X_test.csv')
X_val= pd.read_csv('X_val.csv')

In [ ]:
from albumentations import (
PadIfNeeded,
HorizontalFlip,
VerticalFlip,
Transpose,
HueSaturationValue,
ElasticTransform,
GridDistortion,
OpticalDistortion,
RandomBrightnessContrast,
RandomGamma,Resize
)

In [ ]:
def agument(aug,image,mask):
    """This function will apply the type of augmentation on both image and mask
    returns augmented image and its augmented mask
    """
    augmented = aug(image=image, mask=mask)
    return augmented['image'],augmented['mask']


# All these below functions defines the different agumentations types.

def resize(image,mask): #While agumenting the data we are converting all images into the size of (512,512)
    aug = Resize(height=512,width=512,p=1)
    return agument(aug,image,mask)

def horizontalFlip(image,mask):
    aug = HorizontalFlip(p=1)
    return agument(aug,image,mask)

def verticalFlip(image,mask):
    aug = VerticalFlip(p=1)
    return agument(aug,image,mask)

def transpose(image,mask):
    aug = Transpose(p=1)
    return agument(aug,image,mask)

def hueSaturationValue(image,mask):
    aug = HueSaturationValue(p=1,hue_shift_limit=100, sat_shift_limit=100, val_shift_limit=50)
    return agument(aug,image,mask)

def elasticTransform(image,mask):
    aug = ElasticTransform(p=1)
    return agument(aug,image,mask)

def opticalDistortion(image,mask):
    aug = OpticalDistortion(p=1, distort_limit=3, shift_limit=0.4)
    return agument(aug,image,mask)

def randomBrightnessContrast(image,mask):
    aug =RandomBrightnessContrast(p=1,brightness_limit=0.5, contrast_limit=0.4)
    return agument(aug,image,mask)

In [ ]:
# def resize(image,mask): #While agumenting the data we are converting all images into the size of (512,512)
#     aug = Resize(height=512,width=512,p=1)
#     return agument(aug,image,mask)


In [ ]:
# index=31
# fig, ax = plt.subplots(9,2,figsize=(10,40))
# image_raw= imread('/content/CASIA2/Tp/'+X_train[X_train['label']=='fake']['name'].iloc[index])
# mask_raw = imread('/content/CASIA2/CASIA 2 Groundtruth/'+X_train[X_train['label']=='fake']['name'].iloc[index].split('.')[0]+'_gt.png')

# image,mask= resize(image=image_raw,mask=mask_raw)
# image_h,mask_h= horizontalFlip(image=image,mask=mask)
# image_v,mask_v= verticalFlip(image=image,mask=mask)
# image_t,mask_t= transpose(image=image,mask=mask)
# image_hue,mask_hue= hueSaturationValue(image=image,mask=mask)
# image_elt,mask_elt= elasticTransform(image=image,mask=mask)
# image_opt,mask_opt= opticalDistortion(image=image,mask=mask)
# image_rbc,mask_rbc= randomBrightnessContrast(image=image,mask=mask)

# ax[0][0].imshow(image_raw)
# ax[0][1].imshow(mask_raw,cmap='gray')
# ax[0][0].set_title('Raw FAKE image')
# ax[0][1].set_title('corresponding MASK for this FAKE image')
# ax[1][0].imshow(image)
# ax[1][1].imshow(mask,cmap='gray')
# ax[1][0].set_title('A resized FAKE image')
# ax[1][1].set_title('resized MASK for this FAKE image')
# ax[2][0].imshow(image_h)
# ax[2][1].imshow(mask_h,cmap='gray')
# ax[2][0].set_title('horizontally fliped image')
# ax[2][1].set_title('horizontally fliped mask')
# ax[3][0].imshow(image_v)
# ax[3][1].imshow(mask_v,cmap='gray')
# ax[3][0].set_title('vertically fliped image')
# ax[3][1].set_title('vertically fliped mask')
# ax[4][0].imshow(image_t)
# ax[4][1].imshow(mask_t,cmap='gray')
# ax[4][0].set_title('transposed image')
# ax[4][1].set_title('transposed mask')
# ax[5][0].imshow(image_hue)
# ax[5][1].imshow(mask_hue,cmap='gray')
# ax[5][0].set_title('hue shifted image')
# ax[5][1].set_title('hue shifted mask')
# ax[6][0].imshow(image_elt)
# ax[6][1].imshow(mask_elt,cmap='gray')
# ax[6][0].set_title('elastic_transformed image')
# ax[6][1].set_title('elastic_transformed mask')
# ax[7][0].imshow(image_opt)
# ax[7][1].imshow(mask_opt,cmap='gray')
# ax[7][0].set_title('optical_distorted image')
# ax[7][1].set_title('optical_distorted mask')
# ax[8][0].imshow(image_rbc)
# ax[8][1].imshow(mask_rbc,cmap='gray')
# ax[8][0].set_title('Brightness modified image')
# ax[8][1].set_title('Brightness modified mask')
# plt.show()

In [ ]:
train_path_fk = 'data_agumented/train/fake/'
test_path_fk = 'data_agumented/test/fake/'
val_path_fk = 'data_agumented/val/fake/'
os.makedirs(train_path_fk)
os.makedirs(test_path_fk)
os.makedirs(val_path_fk)

In [ ]:
def DataAgument(split_type,Type):
    """Function will accept the data in "split_type" and name of the split in "Type"
     From the data, it will take only FAKE images and its masks.
    for each fake image:
        1. It will resizes the image and its mask
        2. And apply 6 kinds of agumentations
        3. Finally saves the agumented images and masks

    """
    for name in tqdm(split_type[split_type['label']=='fake']['name']):
        name_path ='/content/CASIA2/Tp/'+str(name)
        image_raw= imread(name_path)
        mask_raw = imread('/content/CASIA2/CASIA 2 Groundtruth/'+name.split('.')[0]+'_gt.png')
        image_0,mask_0= resize(image=image_raw,mask=mask_raw)
        # image_1,mask_1= horizontalFlip(image=image_0,mask=mask_0)
        # image_2,mask_2= verticalFlip(image=image_0,mask=mask_0)
        # image_3,mask_3= transpose(image=image_0,mask=mask_0)
        # image_4,mask_4= hueSaturationValue(image=image_0,mask=mask_0)
        # image_5,mask_5= elasticTransform(image=image_0,mask=mask_0)
        # image_6,mask_6= opticalDistortion(image=image_0,mask=mask_0)
        # image_7,mask_7= randomBrightnessContrast(image=image_0,mask=mask_0)
        cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(0)+'.png',image_0)
        cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(0)+'.mask.png',mask_0)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(1)+'.png',image_1)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(1)+'.mask.png',mask_1)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(2)+'.png',image_2)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(2)+'.mask.png',mask_2)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(3)+'.png',image_3)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(3)+'.mask.png',mask_3)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(4)+'.png',image_4)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(4)+'.mask.png',mask_4)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(5)+'.png',image_5)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(5)+'.mask.png',mask_5)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(6)+'.png',image_6)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(6)+'.mask.png',mask_6)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(7)+'.png',image_7)
        # cv2.imwrite('data_agumented/'+Type+'/fake/'+name.split('.')[0]+'_'+str(7)+'.mask.png',mask_7)


In [ ]:
DataAgument(split_type=X_train,Type="train")
DataAgument(split_type=X_test,Type="test")
DataAgument(split_type=X_val,Type="val")

In [ ]:
print("Total agumented FAKE images from TRIAN Split: {}".format(len(os.listdir(train_path_fk))/2))
print("Total agumented FAKE images from TEST Split: {}".format(len(os.listdir(test_path_fk))/2))
print("Total agumented FAKE images from VALIDATION Split: {}".format(len(os.listdir(val_path_fk))/2))

In [ ]:
train_path_pr = 'data_agumented/train/pristine/'
test_path_pr = 'data_agumented/test/pristine/'
val_path_pr = 'data_agumented/val/pristine/'
os.makedirs(train_path_pr)
os.makedirs(test_path_pr)
os.makedirs(val_path_pr)

In [ ]:
cv2.imwrite("default.mask.png",np.zeros((512,512))+255)

In [ ]:
def DataAgument_pristine(split_type,Type):
    for name in tqdm(split_type[split_type['label']=='pristine']['name']):
        name_path = '/content/CASIA2/Au/'+str(name)
        image_raw= imread(name_path)
        mask_raw= imread('default.mask.png')
        image_0,mask_0= resize(image=image_raw,mask=mask_raw)
        # image_1,mask_1= verticalFlip(image=image_0,mask=mask_raw)
        # #image_2,mask_2= hueSaturationValue(image=image_0,mask=mask_raw)
        # image_2,mask_2= opticalDistortion(image=image_0,mask=mask_raw)
        cv2.imwrite('data_agumented/'+Type+'/pristine/'+name.split('.')[0]+'_'+str(0)+'.png',image_0)
        # cv2.imwrite('data_agumented/'+Type+'/pristine/'+name.split('.')[0]+'_'+str(1)+'.png',image_1)
        # #cv2.imwrite('data_agumented/'+Type+'/pristine/'+name.split('.')[0]+'_'+str(2)+'.png',image_2)
        # cv2.imwrite('data_agumented/'+Type+'/pristine/'+name.split('.')[0]+'_'+str(3)+'.png',image_2)

In [ ]:
DataAgument_pristine(split_type=X_train,Type="train")
DataAgument_pristine(split_type=X_test,Type="test")
DataAgument_pristine(split_type=X_val,Type="val")

In [ ]:
print("Total agumented FAKE images from TRIAN Split: {}".format(len(os.listdir(train_path_fk))/2))
print("Total agumented FAKE images from TEST Split: {}".format(len(os.listdir(test_path_fk))/2))
print("Total agumented FAKE images from VALIDATION Split: {}".format(len(os.listdir(val_path_fk))/2))

In [ ]:
print("Total agumented Pristine images from TRIAN Split: {}".format(len(os.listdir(train_path_pr))))
print("Total agumented Pristine images from TEST Split: {}".format(len(os.listdir(test_path_pr))))
print("Total agumented Pristine images from VALIDATION Split: {}".format(len(os.listdir(val_path_pr))))

In [ ]:
train_path_fk = 'data_agumented/train/fake/'
test_path_fk = 'data_agumented/test/fake/'
val_path_fk = 'data_agumented/val/fake/'
train_path_pr = 'data_agumented/train/pristine/'
test_path_pr = 'data_agumented/test/pristine/'
val_path_pr = 'data_agumented/val/pristine/'

In [ ]:
train_data = pd.DataFrame()
X_label = []
y_label = []
for i in (list({i.split('.')[0] for i in os.listdir(train_path_fk)})):
    X_label.append(train_path_fk+i+'.png')
    y_label.append(train_path_fk+i+'.mask.png')
for i in (list({i.split('.')[0] for i in os.listdir(train_path_pr)})):
    X_label.append(train_path_pr+i+'.png')
    y_label.append('default.mask.png')

train_data['X']  =  X_label
train_data['y']  =  y_label
train_data = shuffle(train_data)
train_data.reset_index(inplace=True, drop=True)
train_data.head()

In [ ]:
print(train_data.shape)
# print(test_data.shape)

In [ ]:
test_data = pd.DataFrame()
X_label = []
y_label = []
for i in (list({i.split('.')[0] for i in os.listdir(test_path_fk)})):
    X_label.append(test_path_fk+i+'.png')
    y_label.append(test_path_fk+i+'.mask.png')
for i in (list({i.split('.')[0] for i in os.listdir(test_path_pr)})):
    X_label.append(test_path_pr+i+'.png')
    y_label.append('default.mask.png')

test_data['X']  =  X_label
test_data['y']  =  y_label
test_data = shuffle(test_data)
test_data.reset_index(inplace=True, drop=True)
test_data.head()

In [ ]:
test_data.shape

In [ ]:
val_data = pd.DataFrame()
X_label = []
y_label = []
for i in (list({i.split('.')[0] for i in os.listdir(val_path_fk)})):
    X_label.append(val_path_fk+i+'.png')
    y_label.append(val_path_fk+i+'.mask.png')
for i in (list({i.split('.')[0] for i in os.listdir(val_path_pr)})):
    X_label.append(val_path_pr+i+'.png')
    y_label.append('default.mask.png')

val_data['X']  =  X_label
val_data['y']  =  y_label
val_data = shuffle(val_data)
val_data.reset_index(inplace=True, drop=True)
val_data.head()

In [ ]:
def data_gen(data):
    while True:
        for start in range(0, data.shape[0], batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, data.shape[0])
            x_data_batch = data.iloc[start:end]

            for index,row in x_data_batch.iterrows():
                image = load_img(row['X'])
                image = img_to_array(image)
                mask = imread(row['y'])
                mask= img_to_array(mask,dtype=np.uint8)
                x_batch.append(image)
                y_batch.append(mask)

            x_batch = np.array(x_batch).reshape(-1, 512, 512, 3)/255
            # temp= np.array(y_batch[0])
            # #print(temp)
            # print(temp.shape)
            # temp.shape = (512,512)
            # print(temp.shape)
            # print(temp)
            temp_arr =[]
            #print(len(y_batch))
            #y_batch = np.array(y_batch)/255
            # for b in y_batch:
            #   print(b)
            #   b.shape = (512,512)
            #   temp_arr.append(b)
            # temp_arr = np.array(temp_arr)/255
            for b in y_batch:
              temp = b[:, :, 0]
              temp_arr.append(temp)
            temp_arr = np.array(temp_arr)/255

            yield x_batch, temp_arr

In [ ]:
batch_size = 16

In [ ]:
listed = list(data_gen(train_data))

In [ ]:
# #Ref : https://www.depends-on-the-definition.com/unet-keras-segmenting-images/

# def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
#     # first layer
#     x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
#                padding="same")(input_tensor)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     # second layer
#     x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
#                padding="same")(x)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation("relu")(x)
#     return x

In [ ]:
# def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
#     # contracting path
#     c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
#     p1 = MaxPooling2D((2, 2)) (c1)
#     p1 = Dropout(dropout*0.5)(p1)

#     c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
#     p2 = MaxPooling2D((2, 2)) (c2)
#     p2 = Dropout(dropout)(p2)

#     c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
#     p3 = MaxPooling2D((2, 2)) (c3)
#     p3 = Dropout(dropout)(p3)

#     c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
#     p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
#     p4 = Dropout(dropout)(p4)

#     c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
#     # expansive path
#     u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
#     u6 = concatenate([u6, c4])
#     u6 = Dropout(dropout)(u6)
#     c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

#     u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
#     u7 = concatenate([u7, c3])
#     u7 = Dropout(dropout)(u7)
#     c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

#     u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
#     u8 = concatenate([u8, c2])
#     u8 = Dropout(dropout)(u8)
#     c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

#     u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
#     u9 = concatenate([u9, c1], axis=3)
#     u9 = Dropout(dropout)(u9)
#     c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)

#     outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
#     model = Model(inputs=[input_img], outputs=[outputs])
#     return model

In [ ]:
def train(batch_norm=False):
    model=Sequential()
    model.add(Conv2D(filters=64,
              kernel_size=(3, 3),
              padding='same',
              activation='relu',
              input_shape=(224,224,3),
              name='conv1_1'))
    model.add(Conv2D(filters=64,
              kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv1_2'))
    model.add(MaxPooling2D(pool_size=(2,2),
                   strides=(2,2),
                   name='max_pooling2d_1'))
    model.add(Conv2D(filters=128,
              kernel_size=(3, 3),
              padding='same',
              activation='relu',
              name='conv2_1'))
    model.add(Conv2D(filters=128,
              kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv2_2'))
    model.add(MaxPooling2D(pool_size=(2,2),
                 strides=(2,2),
                 name='max_pooling2d_2'))
    model.add(Conv2D(filters=256,
              kernel_size=(3, 3),
              padding='same',
              activation='relu',
              input_shape=(224,224,3),
              name='conv3_1'))
    model.add(Conv2D(filters=256,
              kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv3_2'))
    model.add(Conv2D(filters=256,
              kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv3_3'))

    model.add(MaxPooling2D(pool_size=(2,2),
                   strides=(2,2),
                   name='max_pooling2d_3'))
    model.add(Conv2D(filters=512,
              kernel_size=(3, 3),
              padding='same',
              activation='relu',
              input_shape=(224,224,3),
              name='conv4_1'))
    model.add(Conv2D(filters=512,
              kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv4_2'))
    model.add(Conv2D(filters=512,
              kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv4_3'))

    model.add(MaxPooling2D(pool_size=(2,2),
                   strides=(2,2),
                   name='max_pooling2d_4'))
    model.add(Conv2D(filters=512,
              kernel_size=(3, 3),
              padding='same',
              activation='relu',
              input_shape=(224,224,3),
              name='conv5_1'))
    model.add(Conv2D(filters=512,
                  kernel_size=(3, 3),
                  padding='same',
              activation='relu',
              name='conv5_2'))
    model.add(Conv2D(filters=512,
             kernel_size=(3, 3),
                 padding='same',
             activation='relu',
             name='conv5_3'))

    model.add(MaxPooling2D(pool_size=(2,2),
                   strides=(2,2),
                   name='max_pooling2d_5'))
    # model.add(Flatten(name='flatten'))

    # model.add(Dense(4096, activation='relu', name='fc_1'))
    # model.add(Dropout(0.5, name='dropout_1'))

    # model.add(Dense(4096, activation='relu', name='fc_2'))
    # model.add(Dropout(0.5, name='dropout_4'))

    # model.add(Dense(1000, activation='softmax', name='output'))
    # model.add(Conv2D(filters=512,
    #           kernel_size=(3, 3),
    #           padding='same',
    #           activation='relu',
    #           input_shape=(224,224,3),
    #           name='conv5_1'))
    return model

In [ ]:
print(input_img)

In [ ]:
input_img = Input((512, 512, 3))
model_unet = train(input_img )

model_unet.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model_unet.summary()

In [ ]:
checkPoint_path = "model_checkpoints/model_unet/"
filepath= checkPoint_path+"weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

log_dir = "C:\\logs\\model_unet\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=3)

callbacks_list = [checkpoint,tensorboard_callback]

In [ ]:
train_data

In [ ]:
ans = tf.keras.backend.is_keras_tensor(
    input_img
)

In [ ]:
tf.keras.backend.image_data_format()

In [ ]:
data_gen(train_data)

In [ ]:
batch_size = 16
max_epochs = 10
input_size = 512

In [ ]:
batch_size = 16
max_epochs = 10
input_size = 512

history = model_unet.fit(x = data_gen(train_data),
                    steps_per_epoch=16,
                    epochs=max_epochs,
                    validation_data=data_gen(val_data),
                    validation_steps=16,
                    callbacks=callbacks_list)

In [ ]:
model_unet.save('trained_models/model_unet.h5')

In [ ]:
test_loss,test_accuracy = model_unet.evaluate_generator(data_gen(test_data),steps=16,verbose=1)
val_loss,val_accuracy = model_unet.evaluate_generator(data_gen(val_data),steps=16,verbose=1)
train_loss,train_accuracy = model_unet.evaluate_generator(data_gen(train_data),steps=16,verbose=1)

In [ ]:
print("Accuracy on TRAIN Data = {} \t Log-loss in TRAIN Data = {} ".format(train_accuracy,train_loss))
print("Accuracy on VALIDATION Data = {} \t Log-loss in VALIDATION Data = {} ".format(val_accuracy,val_loss))
print("Accuracy on TEST Data = {} \t Log-loss in TEST Data = {} ".format(test_accuracy,test_loss))

In [ ]:
def print_results(model_path,split_data_df,show_images):
    """This function prints the input image,its ground truth mask along with predicted mask
        model_path : Takes the path of trianed model
        split_data_df : Takes the pd.DataFrame object that should contain image path and mask path in X and Y named columns
        show_images : number of random images that should sampled from "split_data_df"
    """
    samples = split_data_df.sample(n=show_images)
    model = load_model(model_path)
    input_img = []
    GT_mask = []
    Predicted_mask = []
    for index,row in samples.iterrows():
        image = load_img(row['X'])
        image_arr = img_to_array(image)
        image_arr=np.array(image_arr).reshape(-1, 512, 512, 3)/255
        mask = imread(row['y'])
        output = model.predict(image_arr).reshape(512, 512)
        input_img.append(image)
        GT_mask.append(mask)
        Predicted_mask.append(output)

    for i in range(show_images):
        fig, ax = plt.subplots(1,3,figsize=(15, 10))
        ax[0].imshow(input_img[i])
        ax[0].set_title("Input Image")
        ax[1].imshow(Predicted_mask[i])
        ax[1].set_title("Predicted Mask")
        ax[2].imshow(GT_mask[i])
        ax[2].set_title("Groud Truth Mask")

In [ ]:
# print_results(model_path='trained_models/model_unet.h5',
#               split_data_df =test_data,show_images=10)
print_results(model_path='trained_models/model_unet.h5',
              split_data_df =test_data[test_data['y']!='default.mask.png'],show_images=25)

In [ ]:
test_data.tail()

In [ ]:
def data_gen_fk(data):
    data_reduced = data[data['y']!='default.mask.png']
    while True:
        for start in range(0, data_reduced.shape[0], batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, data_reduced.shape[0])
            x_data_batch = data_reduced.iloc[start:end]

            for index,row in x_data_batch.iterrows():
                image = load_img(row['X'])
                image = img_to_array(image)
                mask = imread(row['y'])
                mask= img_to_array(mask,dtype=np.uint8)
                x_batch.append(image)
                y_batch.append(mask)

            x_batch = np.array(x_batch).reshape(-1, 512, 512, 3)/255
            y_batch = np.array(y_batch)/255

            yield x_batch, y_batch

In [ ]:
checkPoint_path = "model_checkpoints/model_unet_fk/"
filepath= checkPoint_path+"weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

log_dir = "C:\\logs\\model_unet_fk\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=3)

callbacks_list = [checkpoint,tensorboard_callback]

In [ ]:
input_img = Input((512, 512, 3))
model_unet_2 = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)

model_unet_2.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
#model_unet_2.summary()

In [ ]:
batch_size = 16
max_epochs = 10
input_size = 512

history = model_unet_2.fit_generator(generator = data_gen_fk(train_data),
                    steps_per_epoch=16,
                    epochs=max_epochs,
                    validation_data=data_gen_fk(val_data),
                    callbacks=callbacks_list,
                    validation_steps=16)

In [ ]:
model_unet_2.save('trained_models/model_unet_2.h5')


In [ ]:
test_loss_2,test_accuracy_2 = model_unet_2.evaluate_generator(data_gen_fk(test_data),steps=16,verbose=1)
val_loss_2,val_accuracy_2 = model_unet_2.evaluate_generator(data_gen_fk(val_data),steps=16,verbose=1)
train_loss_2,train_accuracy_2 = model_unet_2.evaluate_generator(data_gen_fk(train_data),steps=16,verbose=1)


In [ ]:
print("Accuracy on TRAIN Data = {} \t Log-loss in TRAIN Data = {} ".format(train_accuracy_2,train_loss_2))
print("Accuracy on VALIDATION Data = {} \t Log-loss in VALIDATION Data = {} ".format(val_accuracy_2,val_loss_2))
print("Accuracy on TEST Data = {} \t Log-loss in TEST Data = {} ".format(test_accuracy_2,test_loss_2))

In [ ]:
print_results(model_path='trained_models/model_unet.h5',
              split_data_df =test_data[test_data['y']!='default.mask.png'],show_images=25)

print_results(model_path='trained_models/model_unet.h5',
              split_data_df =test_data,show_images=10)

In [ ]:
import requests
from bs4 import BeautifulSoup

def analyze_traffic(url):
  """
  Analyzes website traffic for phishing indicators.

  Args:
    url: The website URL to analyze.

  Returns:
    Dictionary containing:
      - total_traffic: Estimated total monthly traffic.
      - traffic_sources: Dictionary of traffic sources and their percentages.
      - phishing_indicators: List of potential phishing indicators.
  """

  # Fetch website content using requests
  response = requests.get(url)
  content = response.content

  # Extract relevant data using BeautifulSoup
  soup = BeautifulSoup(content, "html.parser")

  # Initialize data
  traffic_data = {}
  traffic_sources = {}
  phishing_indicators = []

  # Analyze traffic based on available data (example)
  try:
    # Estimate total traffic using external services (optional)
    # ...

    # Look for suspicious traffic sources
    for link in soup.find_all("a"):
      href = link.get("href")
      if href and not href.startswith(url):
        source_domain = urlparse(href).netloc
        if source_domain not in traffic_sources:
          traffic_sources[source_domain] = 0
        traffic_sources[source_domain] += 1

    # Check for common phishing indicators
    if any(word in soup.text for word in ["free", "money", "win", "prize", "urgent"]):
      phishing_indicators.append("Suspicious keywords")

    # ... (add more indicators)

    traffic_data = {
      "total_traffic": None,  # replace with actual value
      "traffic_sources": traffic_sources,
      "phishing_indicators": phishing_indicators,
    }
  except Exception as e:
    print(f"Error analyzing traffic for {url}: {e}")

  return traffic_data

# Example usage
url = "https://www.example.com"
traffic_data = analyze_traffic(url)

print(f"Total traffic: {traffic_data['total_traffic']}")
print(f"Traffic sources: {traffic_data['traffic_sources']}")
print(f"Phishing indicators: {traffic_data['phishing_indicators']}")
